# Six-bar mechanism balancing

###### Luis Eduardo Robles Jiménez

In [ ]:
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

### Read data from a file

In [ ]:
"""
# The very best examples, to understand how good solutions behave
fitness = np.loadtxt("1671649703Fitness.txt")
samples = np.loadtxt("1671649703Population.txt")
# Solutions under 1.0
fitness = np.loadtxt("1671919014Fitness.txt")
samples = np.loadtxt("1671919014Population.txt")
# Broader space
# 10% best solutions, 50% Under 1.0 solutions, 50% random solutions.
samples = np.loadtxt("1671988185Population.txt")
fitness = np.loadtxt("1671988185Fitness.txt")
shForces = np.loadtxt("1671988185ShForces.txt")
shMoments = np.loadtxt("1671988185ShMoments.txt")
# More pymoo results
samples = np.loadtxt("1674756309Population.txt")
fitness = np.loadtxt("1674756309Fitness.txt")
shForces = np.loadtxt("1674756309ShForces.txt")
shMoments = np.loadtxt("1674756309ShMoments.txt")
# 114 crazy estimations with a random uniform alpha (0 pairs below 1)
samples = np.loadtxt("1675557893Population.txt")
fitness = np.loadtxt("1675557893Fitness.txt")
shForces = np.loadtxt("1675557893ShForces.txt")
shMoments = np.loadtxt("1675557893ShMoments.txt")
# DE with random alpha
samples = np.loadtxt("experiments/1676302198Population.txt")
fitness = np.loadtxt("experiments/1676302198Fitness.txt")
shForces = np.loadtxt("experiments/1676302198ShForces.txt")
shMoments = np.loadtxt("experiments/1676302198ShMoments.txt")
# Pymoo
samples = np.loadtxt("experiments/1676338610Population.txt")
fitness = np.loadtxt("experiments/1676338610Fitness.txt")
shForces = np.loadtxt("experiments/1676338610ShForces.txt")
shMoments = np.loadtxt("experiments/1676338610ShMoments.txt")
"""
"""
"""
# Pymoo 3 weights
samples = np.loadtxt("experiments/1677893800Population.txt")
fitness = np.loadtxt("experiments/1677893800Fitness.txt")
shForces = np.loadtxt("experiments/1677893800ShForces.txt")
shMoments = np.loadtxt("experiments/1677893800ShMoments.txt")

print(samples.shape)
print(fitness.shape)
print(shForces.shape)
print(shMoments.shape)

In [ ]:
nSamples, nVariables = samples.shape
nWeights = nVariables // 3
globalMax = np.max(samples)
globalMin = np.min(samples)

## Dimensionality reduction

### Feature extraction

#### Thicknesses

In [ ]:
thicknesses = samples[:, 2::3]

#### Radiuses

In [ ]:
radiuses = np.sqrt(np.square(samples[:, ::3]) + np.square(samples[:, 1::3]))

#### Masses

This is a problem-specific feature that can be got from variables.

These calculations are intended to be used later on to find out whether this is a good representant of the solutions or not.

The mass can be easily calculated with the resulting formula of the following steps.

##### Formula

$$\sigma_i = \frac{m_i}{V_i}$$

$$m_i = V_i \sigma_i$$

$$m_i = (r_i^2 t_i \pi)(\sigma_i)$$

$$m_i = (\sqrt{x_i^2 + y_i^2}^2 t_i \pi)(\sigma_i)$$

$$m_i = (x_i^2 + y_i^2) t_i \pi\sigma_i$$

##### Considering that

$\sigma$ is density. In this problem: $\sigma_i = 7800 \frac{kg}{m^3}$

$m$ is mass $(kg)$

$V$ is volume; of a cylinder in this problem $(m^3)$ 

$x, y$ are the relative coordinates of the weight centers. $(m)$

In [ ]:
masses = np.zeros((nSamples, nWeights))
density = 7800
for i in range(nWeights):
    masses[:, i] = (samples[:, i*3]**2 + samples[:, i*3 + 1]**2) * samples[:, i*3 + 2] * np.pi * density
sumMasses = np.sum(masses, axis = 1)

#### Alpha

Alpha can be calculated out of the fitness function.

Does this method work if we apply this formula even for algorithms that didn't use this method?

- First problem: the resultant alpha is not standardised [0, 1]

##### Formula

$$
f(x) = \alpha ShF(x) + (1 - \alpha) ShM(x)
$$
$$
\therefore \alpha = \frac{f(x) - ShM(x)}{ShF(x) - ShM(x)}
$$

In [ ]:
alpha = (fitness - shMoments)/(shForces - shMoments)
print(np.max(alpha))
print(np.min(alpha))    

#### PCA

In [ ]:
modelPCA = PCA()
modelPCA.fit(samples)
dataPCA = modelPCA.transform(samples)
dataPCA.shape

In [ ]:
expVar = modelPCA.explained_variance_ratio_
expVarCum = np.cumsum(expVar)
plt.figure(figsize=(10, 5))
plt.grid()
plt.plot(np.concatenate(([0], expVarCum)))
plt.show()

##### First two main components

In [ ]:
dataPCA = dataPCA[:, :2]
plt.figure(figsize=(10, 10))
plt.title("PCA first two components")
plt.scatter(dataPCA[:, 0], dataPCA[:, 1])
plt.xlabel("PCA 1 -> " + str(np.round(expVar[0]*100, 2)) + "%")
plt.ylabel("PCA 2 -> " + str(np.round(expVar[1]*100, 2)) + "%")
plt.show()

## Data visualization

### Parallel coordinates 

#### All variables

Decent solutions seem to follow a pattern of trying to find the equilibrium between thickness and position. So the weights might be optimal with a determined mass value.

In [ ]:
figure = []
features = ["x One", "y One", "t One", "x Two", "y Two", "t Two", "x Three", "y Three", "t Three", "x Four", "y Four", "t Four", "x Five", "y Five", "t Five"]
for i in range(samples.shape[1]):
    figure.append(dict(range = [globalMin, globalMax], label = features[i], values = samples[:, i]))
    #figure.append(dict(label = features[i], values = samples[:, i]))

In [ ]:
fig = go.Figure(data =
    go.Parcoords(
        line = dict(color = alpha, colorscale = px.colors.sequential.Jet),
        dimensions = figure
    )
)
fig.show()

#### Masses

The masses seem to describe a consistent behavior on their weights. 

In [ ]:
figure = []
features = ["mass One", "mass Two", "mass Three", "mass Four", "mass Five"]
massMin = np.min(masses)
massMax = np.max(masses)
for i in range(masses.shape[1]):
    figure.append(dict(range = [massMin, massMax], label = features[i], values = masses[:, i]))
    #figure.append(dict(label = features[i], values = masses[:, i]))

In [ ]:
fig = go.Figure(data =
    go.Parcoords(
        line = dict(color = alpha, colorscale = px.colors.sequential.Jet),
        dimensions = figure
    )
)
fig.show()

#### Shaking moment vs Shaking force

This graph shows the behavior of the Pareto efficient solutions, where the improvement on the optimization of one variable, directly affects others'.

In [ ]:
figure = []
figure.append(dict(range = [0, 1], label = "Shaking Force", values = shForces))
figure.append(dict(range = [0, 1], label = "Shaking Moment", values = shMoments))
figure.append(dict(range = [0, 1], label = "Alpha", values = alpha))

In [ ]:
fig = go.Figure(data =
    go.Parcoords(
        line = dict(color = alpha, colorscale = px.colors.sequential.Jet),
        dimensions = figure
    )
)
fig.show()

### Violinplot

This graph is created with hopes of seeing if  masses have a tendency. 

These graphs help to show:

1. Weight 1 has a short range where it values are optimal, and the frequency seems to indicate that the smaller it is, the better it works.

2. Weight 2 shows an interesting dynamic range, it's large and might ressemble a normal distribution.

3. Weight 3 clearly shows it's tendency for a specific mass near the maximum limit.

4. Weight 4 seems optimal when it's as small as possible.

5. Weight 5 same behavior as 4.

In [ ]:
fig = plt.figure(figsize =(15, 15))
plt.title("Distribution of the optimal weights in kg")
plt.xlabel("# of weight")
plt.ylabel("Mass (kg)")
plt.violinplot(masses)

### Scatter plot

Let's try to see if there's a relation between the masses of the weights and their fitness.

#### Observations

- Fwiw, points above the optimal ones mean that the results can be heavily affected by other variables.

- Checkout the previous [results](https://github.com/LuisR-jpg/School/blob/af30b7fbc78e681a47fba3b86f7624ddf7c9c180/Courses%20and%20Miscellaneous/Research/GraphicalAnalysis.ipynb)

In [ ]:
fig = plt.figure(figsize =(15, 30))
for i in range(nWeights):
    plt.subplot(5, 1, i + 1)
    plt.title("Weight " + str(i + 1))
    plt.scatter(masses[:, i], fitness, alpha = 0.5, c = alpha, cmap = 'jet')
    plt.xlabel("Mass")
    plt.ylabel("Alpha")

In [ ]:
fig = plt.figure(figsize =(10, 10))
colors = ['k', 'b', 'g', 'r', 'lime']
for i in range(nWeights):
    plt.title("Weights")
    plt.scatter(masses[:, i], fitness, alpha = 0.6, color = colors[i], label = "Weight " + str(i + 1))
    plt.xlabel("Mass")
    plt.ylabel("Fitness")
plt.legend()

#### Total mass

It's possible to see how worse the solutions get with weights that are outside a determined range.

In [ ]:
plt.figure(figsize=(15, 7))
plt.scatter(sumMasses, alpha, alpha = 0.5, color = '0')
plt.title("Behavior of total mass")
plt.xlabel("Total mass")
plt.ylabel("Alpha")

#### Relation between thickness and radius

- Once again, it demonstrates that these two variables work together to reach the optimal mass.

In [ ]:
plt.figure(figsize=(15, 15))
for w in range(1, nWeights + 1):
    plt.subplot(3, 2, w)
    weight = w - 1
    plt.scatter(radiuses[:, weight], thicknesses[:, weight], c = alpha, alpha = 0.5, cmap = "jet")
    plt.xlabel("Radius " + str(w))
    plt.ylabel("Thickness " + str(w))
plt.subplot(3, 2, 6)
plt.xlabel("Radius " + str(w))
plt.ylabel("Thickness " + str(w))
for w in range(nWeights): plt.scatter(radiuses[:, w], thicknesses[:, w], c = colors[w], alpha = 0.5, cmap = "jet")

#### Fitness 

This feature is the average of both functions

In [ ]:
plt.figure(figsize = (15, 7))
plt.plot(range(samples.shape[0]), shForces, c = 'r', alpha = 0.5)
plt.scatter(range(samples.shape[0]), shForces, c = 'r', alpha = 0.5)
plt.plot(range(samples.shape[0]), alpha, c = "0")
plt.scatter(range(samples.shape[0]), alpha, c = "0")
plt.plot(range(samples.shape[0]), shMoments, c = 'b', alpha = 0.5)
plt.scatter(range(samples.shape[0]), shMoments, c = 'b', alpha = 0.5)
plt.grid(which="both")

#### Pareto front

A more common way to show the Pareto front.

In [ ]:
plt.figure(figsize = (7, 7))
plt.scatter(shForces, shMoments, alpha = 0.5, c = alpha, cmap = "jet", label = "Alpha")
plt.title("Optimal pareto front")
plt.xlabel("Shaking Forces")
plt.ylabel("Shaking Moment")
plt.colorbar()

In [ ]:
plt.scatter(alpha, shForces)
plt.scatter(alpha, shMoments)

## Comparison

### Pareto Front comparison

In [ ]:
# Pymoo
fitnessTwo = np.loadtxt("experiments/1676338610Fitness.txt")
shForcesTwo = np.loadtxt("experiments/1676338610ShForces.txt")
shMomentsTwo = np.loadtxt("experiments/1676338610ShMoments.txt")

plt.figure(figsize = (15, 7.5))

plt.subplot(1, 2, 1)
plt.xlim(0, np.max([np.max([shForces]), np.max([shForcesTwo])]))
plt.ylim(0, np.max([np.max([shMoments]), np.max([shMomentsTwo])]))
plt.scatter(shForces, shMoments, alpha = 0.7, c = alpha, cmap = "jet")
plt.xlabel("Shaking Forces")
plt.ylabel("Shaking Moment")
plt.title("Current Pareto Front")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.xlim(0, np.max([np.max([shForces]), np.max([shForcesTwo])]))
plt.ylim(0, np.max([np.max([shMoments]), np.max([shMomentsTwo])]))
plt.scatter(shForcesTwo, shMomentsTwo, alpha = 0.7, cmap = "jet")
plt.xlabel("Shaking Forces")
plt.ylabel("Shaking Moment")
plt.title("Comparison Pareto Front")

In [ ]:
plt.figure(figsize = (7, 7))
padding = 0.05
plt.xlim(np.min([np.min([shForces]), np.min([shForcesTwo])]) - padding, np.max([np.max([shForces]), np.max([shForcesTwo])]) + padding)
plt.ylim(np.min([np.min([shMoments]), np.min([shMomentsTwo])]) - padding, np.max([np.max([shMoments]), np.max([shMomentsTwo])]) + padding)
plt.scatter(shForcesTwo, shMomentsTwo, marker="x", label = "NSGA-II")
#plt.scatter(shForcesTwo, shMomentsTwo)
plt.scatter(shForces, shMoments, marker = 'o', label = "Differential Evolution")
plt.xlabel("Shaking Forces")
plt.ylabel("Shaking Moment")
plt.title("Comparison Pareto Front")
plt.legend()

### Variables comparison

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
#samples

In [ ]:
df = pd.DataFrame(masses, columns = [f'Weight {i + 1}' for i in range(nWeights)])

In [ ]:
sns.pairplot(df, kind = "reg", diag_kind = "kde")

In [ ]:
sns.pairplot(df)

In [ ]:
from BetaShF import *
from BetaShM import *

In [ ]:
def objective(s, ShF, ShM, alpha = 0.5):
    assert len(s) == 15
    return (alpha)*ShF(s) + (1 - alpha)*ShM(s)

In [ ]:
# t1 vs t2
index = np.random.randint(len(samples))
rango = 0.5
s = samples[index]
divs = 50
t1, t2 = s[2], s[2]
t1 = np.linspace(t1 - rango, t1 + rango, divs)
t2 = np.linspace(t2 - rango, t2 + rango, divs)
t1 = np.linspace(0.005, 0.04, divs)
t2 = np.linspace(0.005, 0.04, divs)
#t1 = np.linspace(0, 1, divs)
#t2 = np.linspace(-1, 0, divs)

In [ ]:
X, Y = np.meshgrid(t1, t2)

In [ ]:
Z = np.zeros((divs, divs))
for i in range(divs):
    for j in range(divs):
        print(f'\rCurrently running {i}, {j}', end = "")
        test = np.copy(samples[index])
        test[2] = X[i, j]
        test[5] = Y[i, j]
        test = np.array(list(test) + [0] * 6)
        Z[i, j] = objective(test, ShF, ShM)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, Z, 50, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

In [42]:
@widgets.interact(angle1 = (-180, 180), angle2 = (-180, 180))
def plot(angle1=0, angle2=0):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.contour3D(X, Y, Z, 50, cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.view_init(angle1, angle2)

interactive(children=(IntSlider(value=0, description='angle1', max=180, min=-180), IntSlider(value=0, descript…